In [2]:
import os
import requests
from dotenv import load_dotenv
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.checkpoint.memory import MemorySaver

# 加载 .env 文件
load_dotenv()

@tool
def get_weather(city: str) -> str:
    """查询指定城市的实时天气"""
    api_key = os.getenv("OPENWEATHER_API_KEY")  # 从 .env 读取
    if not api_key:
        return "没有找到 API Key，请检查 .env 文件是否配置了 OPENWEATHER_API_KEY"

    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&lang=zh_cn&units=metric"

    try:
        response = requests.get(url)
        data = response.json()

        if response.status_code != 200:
            return f"无法获取 {city} 的天气信息: {data.get('message', '未知错误')}"

        desc = data["weather"][0]["description"]   # 天气描述
        temp = data["main"]["temp"]               # 当前温度
        return f"{city} 当前天气：{desc}，气温 {temp}°C"

    except Exception as e:
        return f"查询 {city} 天气时出错: {str(e)}"

# 初始化模型和记忆
llm = ChatOpenAI(model="gpt-4o-mini")
memory = MemorySaver()

# 创建 Agent
agent = create_agent(
    model=llm,
    tools=[get_weather],
    system_prompt="你是一个天气助手。",
    checkpointer=memory,
)

# 调用 Agent
result = agent.invoke(
    {"messages": [("user", "蒙特利尔天气怎么样？")]},
    config={"configurable": {"thread_id": "t1"}},
)

# 打印最终回答
final_answer = result["messages"][-1].content
print(final_answer)


蒙特利尔当前天气：阴，多云，气温 -12.76°C。请注意保暖！
